In [2]:
# Word tokenization
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

text = """When learning data science, you shouldn't get discouraged!
Challenges and setbacks aren't failures, they're just part of the journey. You've got this!"""

#  "nlp" Object is used to create documents with linguistic annotations.
doc = nlp(text)

# Create list of word tokens
token_list = []
for token in doc:
    token_list.append(token.text)
print(token_list)



['When', 'learning', 'data', 'science', ',', 'you', 'should', "n't", 'get', 'discouraged', '!', '\n', 'Challenges', 'and', 'setbacks', 'are', "n't", 'failures', ',', 'they', "'re", 'just', 'part', 'of', 'the', 'journey', '.', 'You', "'ve", 'got', 'this', '!']


In [3]:
from spacy.lang.en import stop_words
spacy_stopwords = stop_words.STOP_WORDS
print('First ten stop words: %s' % list(spacy_stopwords)[:10])

First ten stop words: ['thereupon', 'their', 'with', 'might', 'the', "'ve", 'eleven', 'whose', 'whereafter', 'out']


In [4]:
import en_core_web_sm
# load en_core_web_sm of English for vocabluary, syntax & entities
nlp = en_core_web_sm.load()
docs = nlp(u"All is well that ends well.")
for word in docs:
    print(word.text,word.pos_)

All DET
is AUX
well ADJ
that DET
ends VERB
well ADV
. PUNCT


In [5]:
from spacy import displacy

nytimes= nlp(u"""New York City on Tuesday declared a public health emergency and ordered mandatory measles vaccinations amid an outbreak, becoming the latest national flash point over refusals to inoculate against dangerous diseases.

At least 285 people have contracted measles in the city since September, mostly in Brooklyn’s Williamsburg neighborhood. The order covers four Zip codes there, Mayor Bill de Blasio (D) said Tuesday.

The mandate orders all unvaccinated people in the area, including a concentration of Orthodox Jews, to receive inoculations, including for children as young as 6 months old. Anyone who resists could be fined up to $1,000.""")

entities=[(i, i.label_, i.label) for i in nytimes.ents]
entities
# visual
# displacy.render(nytimes, style = "ent",jupyter = True)

[(New York City, 'GPE', 384),
 (Tuesday, 'DATE', 391),
 (At least 285, 'CARDINAL', 397),
 (September, 'DATE', 391),
 (Brooklyn, 'GPE', 384),
 (Williamsburg, 'GPE', 384),
 (four, 'CARDINAL', 397),
 (Bill de Blasio, 'PERSON', 380),
 (Tuesday, 'DATE', 391),
 (Orthodox Jews, 'PERSON', 380),
 (6 months old, 'DATE', 391),
 (up to $1,000, 'MONEY', 394)]

In [6]:
!python -m spacy download en_core_web_md
import en_core_web_md



✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [23]:
import os
import pandas as pd
from collections import Counter
from spacy import explain
# Uncomment below to use the medium size model, for production (possibly?)
# nlp = en_core_web_md.load()
nlp = en_core_web_sm.load()

# load the text or book in spacy.tokens.doc.Doc object
doc = open(os.getcwd() +'/datasets/secretagent.txt').read()

# use small sample
sentences = nlp(doc)
print('total tokens', len(doc))

entities=[(i, i.label_, i.label) for i in sentences.ents]

#print(entities[:20])




total tokens 541226
[(Gutenberg eBook, 'PERSON', 380), (The Secret Agent, 'WORK_OF_ART', 388), (Joseph Conrad, 'PERSON', 380), (eBook, 'ORG', 383), (eBook, 'ORG', 383), (The Secret Agent
       , 'WORK_OF_ART', 388), (Joseph Conrad



, 'PERSON', 380), (December 24, 2010, 'DATE', 391), (974, 'MONEY', 394), (First, 'ORDINAL', 396), (June 28, 1997, 'DATE', 391), (English, 'LANGUAGE', 389), (UTF-8, 'GPE', 384), (Transcribed, 'PERSON', 380), (1907, 'DATE', 391), (Methuen & Co, 'ORG', 383), (David Price, 'PERSON', 380), (AGENT, 'ORG', 383), (SIMPLE, 'ORG', 383), (JOSEPH CONRAD, 'PERSON', 380)]
PLACES {'wheezy', 'this karl yundt', 'somewhere', 'victoria', 'inspector heat', 'drunks', 'thou', 'grit', 'observatory', 'mrs verloc', 'murder', 'karl yundt', 'went', 'these f. p.', 'an m. p.', 'greenwich park', 'moser', 'mr verloc’s', 'whitehall', 'lewisham', 'states', 'england', 'mumbled', 'transcribed', 'gloomy dampness', 'gutenberg ebook', 'dingy white', 'winnie', 'ill', 'green park', 'utf-8', 'pa

In [27]:
# very crude list of possible placenames
places = list(map(lambda x:x[0], filter(lambda x: (x[1] in ['GPE','PERSON']), entities)))

## normalise these strings
places = list(set(map(lambda x:x.orth_.strip().lower(), places)))

labels = [x.label_ for x in sentences.ents]
#print(Counter(labels))

# useful to refer to for the various labels of entities
for x in Counter(labels).keys():
    print(x, explain(x))

PLACES ['wheezy', 'this karl yundt', 'somewhere', 'victoria', 'inspector heat', 'drunks', 'thou', 'grit', 'observatory', 'mrs verloc', 'murder', 'karl yundt', 'went', 'these f. p.', 'an m. p.', 'greenwich park', 'moser', 'mr verloc’s', 'whitehall', 'lewisham', 'states', 'england', 'mumbled', 'transcribed', 'gloomy dampness', 'gutenberg ebook', 'dingy white', 'winnie', 'ill', 'green park', 'utf-8', 'paris', 'inch', 'the f. p.', 'wherein', 'embassy', 'stevie', 'heat', 'america', 'lausanne', 'alfred wallace', 'apostle', 'girls', 'stott-wartenheim', 'strand', 'alexander ossipon', 'verloc', 'logical', 'spectre', 'artists', 'headlong', 'hygiene', 'somebody', 'f. p.', 'fertile', 'street lodge', 'india', 'aloud', 'commissioners', 'st stephen’s', 'mr vladimir’s', 'the winnie of the belgravian', 'ard', 'brussels', 'madness', 'latorre', 'nevertheless', 'verlocs', 'vladimir', 'stephen', 'outward deference', 'ethelred', 'joseph conrad', 'china bowl', 'caligula', 'sicily', 'this mr vladimir', 'winn'

In [74]:
# lookup places and plot if they exist. Keep within bounding box of europe somehow.
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from pprint import pprint
'''
rubric:
if an individual search takes longer than n seconds,
it’s not really a city name, and the script will thus 
weed out non-names from the final map.
'''
n_wait=2
geolocator = Nominatim(timeout=n_wait, user_agent="book_location_resolver") # OSM service
 
lat_lon = []
valid = []

bounds_uk = [-10.8544921875, 49.82380908513249,2.021484375,59.478568831926395] # UK
bounds_ukse = [-10.8544921875, 49.82380908513249,2.021484375,59.478568831926395] # UK
bounds_g_london = [-0.489,51.28,0.236,51.686]
for idx, original_text in enumerate(places[:1000]): 
    # do not hammer the service:
    if ( (idx > 0) and (0 == (idx % 10)) ): 
        # delay
        time.sleep(12)
    
    try:
        # TODO: can in fact in API use viewbox and bounded=1, viewbox=<x1>,<y1>,<x2>,<y2>
        location = geolocator.geocode(original_text)        
        
        if location:
            if (bounds_g_london[1] < location.latitude < bounds_g_london[3]) and (bounds_g_london[0] < location.longitude < bounds_g_london[2]): 
                # original_text also added
                #location.setattr('raw') = original_text
                
                lat_lon.append(location)
                valid.append(original_text)
                #print(lat_lon)
                # print out those locations that are within our bounding box
                #print(location.latitude, location.longitude)
                #print(dir(location))
                #print(location.raw)
                
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input %s with message %s"% (original_text, e))


df = pd.DataFrame(lat_lon, columns=['Place Name', 'Coordinates'])
df['Original Text'] = valid # add a column of our valid original text queries for the geocoder
file_name = 'retrieved_locations.csv'
df.to_csv(file_name, sep='\t', encoding='utf-8', index=False)
df.head(50)


,Place Name,Coordinates,Original Text
0,"Greenwich Park, East Greenwich, Royal Borough ...","(51.47722835, 0.00092473393014189)",greenwich park
1,"Whitehall, City of Westminster, London, Greate...","(51.5023278, -0.1260826)",whitehall
2,"Lewisham, London, Greater London, England, SE1...","(51.4624325, -0.0101331)",lewisham
3,"Green Park, Piccadilly, St. James's, Mayfair, ...","(51.5066192, -0.1429113)",green park
4,"Kids Aloud Day Care Nursery, St Mary's Road, T...","(51.55991915, -0.00814700415048746)",aloud
5,"Ethelred Estate, Vauxhall, London Borough of L...","(51.49205105, -0.114792931426394)",ethelred
6,"Tottenham, London Borough of Haringey, London,...","(51.5976955, -0.0672892)",tottenham
7,"Greenwich, London, Greater London, England, SE...","(51.4820845, -0.0045417)",greenwich
8,"London, Greater London, England, SW1A 2DX, Uni...","(51.5073219, -0.1276474)",london
9,"Knightsbridge, City of Westminster, London, Gr...","(51.5008444, -0.166965)",knightsbridge


In [81]:
import folium
import geopandas as gpd

def generateBaseMap(default_location=[51.5, 0], default_zoom_start=10):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

def add_marker(place, coordinates, text=""):
    ''' add a location marker to a folium map 
    '''
    folium.Marker(location=list(coordinates), popup=place, icon=folium.DivIcon(
        icon_size=(150,36),
        icon_anchor=(7,20),
        html='<div style="font-size: 12pt; color : #222;text-decoration:underline">'+ text +'</div>',
        )).add_to(base_map)
    
base_map = generateBaseMap()

result = [add_marker(x, y, z) for x, y, z in zip(df['Place Name'], df['Coordinates'], df['Original Text'])]
base_map # display the map